# 範例 16：使用 Ollama 建立自訂模型

Ollama 支援透過 Modelfile 建立自訂模型！

## 重要說明

⚠️ **這不是真正的 Fine-Tuning**

這個範例展示的是透過**系統提示詞**來「定制」模型行為，而不是真正的權重訓練。

| 方式 | 說明 | 效果 |
|------|------|------|
| Modelfile | 設定系統提示詞和參數 | 改變行為風格 |
| Fine-Tuning | 用資料訓練模型權重 | 學習新知識 |

## 學習目標
- 了解 Ollama Modelfile 的結構
- 學會建立自訂模型
- 認識模型參數設定

## 前置需求
- 已安裝 Ollama

## Step 1: 匯入套件

In [ ]:
import subprocess
import os

## Step 2: 了解 Modelfile 結構

Modelfile 是 Ollama 用來定義模型設定的檔案，類似 Dockerfile。

### 基本結構：

```
FROM <基礎模型>

SYSTEM "<系統提示詞>"

PARAMETER <參數名稱> <參數值>
```

In [ ]:
def create_modelfile(base_model, system_prompt, model_name, **params):
    """
    建立 Ollama Modelfile

    參數：
        base_model: 基礎模型名稱
        system_prompt: 系統提示詞
        model_name: 新模型名稱
        **params: 其他參數（如 temperature, top_p 等）
    """
    
    # 基本內容
    modelfile_content = f'''FROM {base_model}

SYSTEM """
{system_prompt}
"""
'''
    
    # 加入參數
    default_params = {
        'temperature': 0.7,
        'top_p': 0.9
    }
    default_params.update(params)
    
    for param, value in default_params.items():
        modelfile_content += f'PARAMETER {param} {value}\n'

    # 儲存 Modelfile
    modelfile_path = f"Modelfile_{model_name}"
    with open(modelfile_path, "w", encoding="utf-8") as f:
        f.write(modelfile_content)

    print(f"已建立 Modelfile: {modelfile_path}")
    return modelfile_path, modelfile_content

## Step 3: 建立程式教學助手模型

In [ ]:
# 定義系統提示詞
programming_teacher_prompt = """你是一位親切的程式設計教師，專門教導初學者學習程式。

你的特點：
- 使用簡單易懂的語言解釋概念
- 提供大量的程式碼範例
- 用生活中的例子來比喻抽象概念
- 鼓勵學生，保持正向態度
- 如果學生犯錯，耐心解釋錯誤原因
- 使用繁體中文回答

回答格式：
1. 先簡單解釋概念
2. 提供程式碼範例
3. 解釋程式碼的每個部分
4. 給予練習建議"""

# 建立 Modelfile
modelfile_path, content = create_modelfile(
    base_model="gpt-oss:120b",
    system_prompt=programming_teacher_prompt,
    model_name="programming-teacher"
)

In [ ]:
# 查看 Modelfile 內容
print("=" * 50)
print("Modelfile 內容：")
print("=" * 50)
print(content)

## Step 4: 建立其他角色模型

In [ ]:
# 英文老師
english_teacher_prompt = """你是一位專業的英文老師。

你的特點：
- 用繁體中文解釋英文文法和用法
- 提供豐富的例句
- 解釋單字的詞源和記憶技巧
- 糾正常見的英文錯誤
- 鼓勵學生多練習口說和寫作"""

create_modelfile(
    base_model="gpt-oss:120b",
    system_prompt=english_teacher_prompt,
    model_name="english-teacher",
    temperature=0.5  # 較低的溫度，回答更一致
)

print()

In [ ]:
# 創意寫作助手
creative_writer_prompt = """你是一位富有想像力的創意寫作助手。

你的特點：
- 擅長各種文體：故事、詩歌、散文、劇本
- 能根據主題發揮創意
- 文字優美、富有感情
- 可以幫助修改和潤飾文章
- 使用繁體中文創作"""

create_modelfile(
    base_model="gpt-oss:120b",
    system_prompt=creative_writer_prompt,
    model_name="creative-writer",
    temperature=0.9,  # 較高的溫度，更有創意
    top_p=0.95
)

## Step 5: 使用 Ollama 建立模型

⚠️ 以下程式碼需要在有 Ollama 的環境中執行

In [ ]:
def create_ollama_model(modelfile_path, model_name):
    """
    使用 Ollama 建立模型

    參數：
        modelfile_path: Modelfile 路徑
        model_name: 新模型名稱
    """
    print(f"正在建立模型 {model_name}...")

    try:
        result = subprocess.run(
            ["ollama", "create", model_name, "-f", modelfile_path],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            print(f"✓ 模型 {model_name} 建立成功！")
            print(f"使用方式：ollama run {model_name}")
            return True
        else:
            print(f"✗ 建立失敗：{result.stderr}")
            return False

    except FileNotFoundError:
        print("找不到 ollama 指令，請確認 Ollama 已安裝")
        return False

In [ ]:
# 顯示手動建立模型的指令
print("=" * 50)
print("要建立模型，請在終端機執行以下指令：")
print("=" * 50)
print("\n1. 程式教學助手：")
print("   ollama create programming-teacher -f Modelfile_programming-teacher")
print("\n2. 英文老師：")
print("   ollama create english-teacher -f Modelfile_english-teacher")
print("\n3. 創意寫作助手：")
print("   ollama create creative-writer -f Modelfile_creative-writer")
print("\n建立完成後，使用以下指令執行：")
print("   ollama run <模型名稱>")

## Modelfile 參數說明

### 常用參數

| 參數 | 說明 | 範圍 | 預設值 |
|------|------|------|--------|
| temperature | 創意程度 | 0-2 | 0.8 |
| top_p | 取樣範圍 | 0-1 | 0.9 |
| top_k | 候選數量 | 1-100 | 40 |
| num_ctx | 上下文長度 | 512-4096+ | 2048 |

### temperature 解釋

```
temperature 低 (0.1-0.3)：回答一致、保守
temperature 中 (0.5-0.7)：平衡創意和一致性
temperature 高 (0.8-1.0)：回答多樣、有創意
```

## Step 6: 進階 Modelfile 功能

In [ ]:
def create_advanced_modelfile(base_model, system_prompt, model_name, 
                              template=None, stop_words=None, **params):
    """
    建立進階 Modelfile，支援更多自訂選項
    """
    
    content = f'FROM {base_model}\n\n'
    
    # 系統提示詞
    content += f'SYSTEM """\n{system_prompt}\n"""\n\n'
    
    # 自訂模板（進階）
    if template:
        content += f'TEMPLATE """\n{template}\n"""\n\n'
    
    # 停止詞
    if stop_words:
        for word in stop_words:
            content += f'PARAMETER stop "{word}"\n'
    
    # 其他參數
    for param, value in params.items():
        content += f'PARAMETER {param} {value}\n'
    
    # 儲存
    filepath = f"Modelfile_{model_name}_advanced"
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    
    return filepath, content

In [ ]:
# 範例：建立一個專注的助手（會在特定標記處停止）
filepath, content = create_advanced_modelfile(
    base_model="gpt-oss:120b",
    system_prompt="你是一個簡潔的助手，回答要精簡扼要。",
    model_name="concise-assistant",
    stop_words=["[END]", "---"],
    temperature=0.5,
    num_ctx=4096
)

print("進階 Modelfile 內容：")
print(content)

## 練習區

In [ ]:
# 建立你自己的自訂模型
# 例如：數學老師、食譜助手、旅遊顧問等

my_system_prompt = """
你是...

你的特點：
- ...
- ...
"""

# 取消註解來建立你的模型
# create_modelfile(
#     base_model="gpt-oss:120b",
#     system_prompt=my_system_prompt,
#     model_name="my-custom-model"
# )

## 重點回顧

1. **Modelfile** 用於定義 Ollama 自訂模型

2. **基本結構**：
   - `FROM`：指定基礎模型
   - `SYSTEM`：設定系統提示詞
   - `PARAMETER`：調整模型參數

3. **這不是 Fine-Tuning**：只是設定提示詞和參數

4. **使用方式**：
   - 建立 Modelfile
   - `ollama create <模型名> -f <Modelfile>`
   - `ollama run <模型名>`

## 下一步

在下一個範例中，我們將學習如何使用 AI 來增強訓練資料！